# Test Assignment 1
Download this video.
Please detect each of the yellow shapes on the video frames. Classify the shapes into classes: circle, rectangle, triangle. Make an output video with each of the shapes signed by the name of its class. Mark the classes with different colors. The shapes which are partly visible may be ignored.

Deliverables for this task:
- code 
- output video

## Import libraries and read video

In [3]:
import numpy as np
import cv2

In [4]:
# Create a video capture object, in this case we are reading the video from a file
cap = cv2.VideoCapture('Test task1_video.mp4')

if (cap.isOpened() == False):
    print("Error opening the video file")
# Read fps and frame count
else:
    # Get frame rate information
    fps = cap.get(5)
    print('Frames per second : ', fps,'FPS')

    # Get frame count
    frame_count = cap.get(7)
    print('Frame count : ', frame_count)
i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    #print(frame.shape) # 1080, 1920
    if ret == True:
        #if(i == 0):
        #    cv2.imwrite('1.jpg',frame)
        #    i = i + 1
        cv2.imshow('Frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

Frames per second :  30.0 FPS
Frame count :  686.0


## Shape detection

In [6]:
import cv2
class ShapeDetector:
    def __init__(self):
        pass
    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
        else:
            shape = "circle"
        # return the name of the shape
        return shape

In [7]:
colors = dict({'triangle': (255, 255, 255), 'square': (255, 0, 255), 'rectangle': (255, 50, 255), 'circle': (0, 255, 255)})

In [8]:
cap = cv2.VideoCapture('Test task1_video.mp4')
sd = ShapeDetector()
i = 0
while(cap.isOpened()):
    # vid_capture.read() methods returns a tuple, first element is a bool 
    # and the second is frame
    ret, frame = cap.read()
    if ret == True:
        # Detect table background and remove it
        f = frame.copy()
        gray = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (7,7), 0)
        f = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
        cnts = cv2.findContours(f, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        contour = cnts[0]
        #cv2.drawContours(frame, [contour], 0, (0, 0, 255), 2)
        
        f = frame.copy()
        cv2.fillPoly(f, [contour], color = (0, 0, 0))
        f = cv2.absdiff(frame, f)
        #if((i % 50 == 0) and (i < 200)):
        #    cv2.imwrite(str(i) + '.jpg',f)
        #print(i)
        #i = i + 1
        hsv = cv2.cvtColor(f, cv2.COLOR_BGR2HSV)
        f = cv2.inRange(hsv, (0, 0, 0), (179,255,183)) # find values for range with a snippet at the end of the file
        

        contours, _ = cv2.findContours(f, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        j = 0
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        for contour in contours:
            # here we are ignoring first counter because
            # findcontour function detects whole image as shape
            if j == 0:
                j = j + 1
                continue
            area = cv2.contourArea(contour)
            if(area > 1500 and area < 200000):
                cv2.drawContours(frame, [contour], 0, (0, 0, 255), 2)
                shape = sd.detect(contour)

                # finding center point of shape
                if(shape != 'undefined'):
                    M = cv2.moments(contour)
                    if M['m00'] != 0.0:
                        x = int(M['m10']/M['m00'])
                        y = int(M['m01']/M['m00'])

                        cv2.putText(frame, shape, (x, y), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=colors[shape], thickness=2)
        
        # Save frame into image file
        s = ""
        if(i < 10):
            s = "00"
        elif(i < 100):
            s = "0"
        cv2.imwrite('res\kang_'+ s +str(i)+'.jpg',frame)
        i = i + 1
        
        # displaying the video
        #cv2.imshow("Processed Video",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
# Save images into a video file
import cv2
import numpy as np
import glob

img_array = []
for filename in glob.glob('res\kang*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

# Snippets

In [ ]:
# Snippet for manually find value for inRange function that looks for yellow shapes in white sheet
import cv2
import numpy as np

def nothing(x):
    pass

# Load image
image = cv2.imread('131.jpg')

# Create a window
cv2.namedWindow('image')

# Create trackbars for color change
# Hue is from 0-179 for Opencv
cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

# Set default value for Max HSV trackbars
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

while(1):
    # Get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin', 'image')
    sMin = cv2.getTrackbarPos('SMin', 'image')
    vMin = cv2.getTrackbarPos('VMin', 'image')
    hMax = cv2.getTrackbarPos('HMax', 'image')
    sMax = cv2.getTrackbarPos('SMax', 'image')
    vMax = cv2.getTrackbarPos('VMax', 'image')

    # Set minimum and maximum HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Convert to HSV format and color threshold
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Print if there is a change in HSV value
    if((phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display result image
    cv2.imshow('image', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()